<a href="https://colab.research.google.com/github/Masfiqur-Mim/LCE-Likehood-based-Copy-Number-Estimation/blob/master/Segmentation_Dual_sequencing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INPUT: Two reads per position file (Reference and Test)
#OUTPUT: Reads Ratio Distribution files with segmentation done over it

In [0]:
from google.colab import drive
drive.mount('/content/drive')

###Assuming that two reads per position of genome file are generated

In [0]:
import csv
import statistics as ss
window_size = 500
reference_file_location = ''
test_file_location = ''
#The test reads per position are to be divided by reference reads per position and to be written in model file
out_file_location = ''
#the normalized distribution's mean and std are saved in model file
model_file_location = ''

reference = open(reference_file_location, 'r')
test = open(test_file_location, 'r')
out = open(out_file_location, 'w')
model = open(model_file_location, 'w')

ref_file_readspp = csv.reader(reference, delimeter= ' ') 
test_file_readspp = csv.reader(test, delimeter= ' ')
reference_genome_size = len(ref_file_readspp) #This will be used while calculating number of windows for initial segmentation
if( len(ref_file_readspp) != len(test_file_readspp) ):
  print("Genome size mismatch!")


out_val = []
for val_ref, val_test in ref_file_readspp, test_file_readspp:
  if(int(val_ref[1] !=0):
    out_val.append(int(val_test[1]) / int(val_ref[1]))
  else:
    out_val.append(int(val_test[1])/1)

out.write("%f\n" % (out_val[i])) for i in range len(out_val)
  



In [0]:
window_reads_ratio_mean = []
window_reads_ratio_std = []
start_window_pos = []
num_windows = int(reference_genome_size/window_size)
start_window_pos.append(i*window_size) for i in range (num_windows)
#if( (reference_genome_size- num_windows*window_size) > 0):
#  start_window_pos.append(num_windows*window_size)
mean_val = 0
std_val = 0
last_window_size = reference_genome_size- num_windows*window_size
for i in range len(num_windows):
  if(num_windows-i==1): #The last window. So it may not be the same size of window_size
    mean_val = sum(out_val[i*window_size:i+last_window_size]) / last_window_size
    std_val = ss.pstdev(out_val[i*window_size:reference_genome_size]) 
  else:#general situation
    mean_val = sum(out_val[i*window_size:(i+1)*window_size]) / window_size
    std_val = ss.pstdev(out_val[i*window_size:(i+1)*window_size]) #pstdev is for actual standard deviation. stdev is for sample standard deviation
  
  window_reads_ratio_mean.append(mean_val)
  window_reads_ratio_std.append(std_val)

##Segmentation Starts

###Window merging function

In [0]:
def merge_windows(mean_list, stdev_list, left_index, start_window):
  n1 = start_window[left_index+1]-start_window[left_index]
  n2 = start_window[left_index+2]-start_window[left_index+1]
  x1 = mean_list[left_index]
  x2 = mean_list[left_index+1] 
  s1 = stdev_list[left_index]
  s2 = stdev_list[left_index+1]
  
  updated_mean = (x1*n1+x2*n2)/(n1+n2)
  
  d1 = x1 - updated_mean
  d2 = x2 - updated_mean

  numerator = n1*(s1*s1+d1*d1)+n2*(s2*s2+d2*d2)
  denominator = (n1+n2)
  updated_stdev = numerator/denominator

  mean_list[left_index] = updated_mean
  stdev_list[left_index] = updated_stdev

  #pop() takes the index as input and removes that item efficiently along with updating the list
  mean_list.pop(left_index+1)
  stdev_list.pop(left_index+1)

###Merge on thresholds

In [0]:
while(p<len(window_reads_mean)):
  if(window_reads_mean[p]+window_reads_std[p]*3 > window_reads_mean[p+1] and window_reads_mean[p]-window_reads_std[p]*3 < window_reads_mean[p+1]):
    merge_windows(window_reads_mean, window_reads_std, p, start_window)
  elif(window_reads_mean[p+1]+window_reads_std[p+1]*3 > window_reads_mean[p] and window_reads_mean[p+1]-window_reads_std[p+1]*3 < window_reads_mean[p]):
    merge_windows(window_reads_mean, window_reads_std, p, start_window)
  else:
    p+=1